In [33]:
import numpy as np
import pandas as pd
import json
import altair as alt
#from vega_datasets import data


#alt.renderers.enable('notebook')
alt.data_transformers.enable('json') 

#alt.renderers.enable('json')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [49]:
#df=pd.read_json('/home/felipe/Github/Trabalho_Vis/Camara_Deputados/df_topic_new4.json')#,error_bad_lines=False)
#df = df[df['Dominant_Topic']!='Votação da Câmara dos Deputados']
#df = df[df['Data']>='2012-01-01'].reset_index()
#url = 'topicos.json'
#df.to_json(url,orient = 'records')

In [54]:
eventos = pd.DataFrame([
    {
        "start": "2014-03-01",
        "end": "2019-12-19",
        "evento": "Lava Jato"
    },
    {
        "start": "2018-04-07",
        "end": "2019-11-08",
        "evento": "Lula é preso"
    },
    {
        "start": "2013-06-02",
        "end": "2013-07-01",
        "evento": "Protestos 2013"
    },
    {
        "start": "2016-04-15",
        "end": "2016-08-25",
        "evento": "Impeachment"
    },
    {
        "start": "2017-05-17",
        "end": "2017-07-17",
        "evento": "Friboigate"
    }
])

In [57]:
pts=alt.selection(type="multi",encodings=['x'])
selection = alt.selection_multi(fields=['Dominant_Topic'], bind='legend')

bar = alt.Chart('data/topicos.json').mark_bar().encode(alt.X('Dominant_Topic:N'),\
                                                        alt.Y('median(Word_Count):Q'),color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))).\
properties(width=1000,height=100).add_selection(pts)

brush = alt.selection(type='interval', encodings=['x'])

#tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]
base = alt.Chart('data/topicos.json').mark_area(interpolate='monotone').encode(
    alt.X('yearmonth(Data):T',axis=alt.Axis(format=("%b %Y"),domain=False,tickSize=0)),
    alt.Y('sum(Word_Count):Q',stack='center'),alt.Color('Dominant_Topic:N',scale=alt.Scale(scheme='dark2'),legend=alt.Legend(orient='right'))
,opacity=alt.condition(selection, alt.value(1), alt.value(0.2))).properties(width=1000,height=400).transform_filter(pts).add_selection(selection)

upper = base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush)),tooltip=['Dominant_Topic:N'])

lower = base.encode(alt.Color('Dominant_Topic:N',scale=alt.Scale(scheme='dark2'),legend=None)).properties(height=60).add_selection(brush)

rule=alt.Chart(eventos).mark_rule(color='black',strokeWidth=1).encode(x='start:T')
text=alt.Chart(eventos).mark_text(align='left',baseline='top',dy=-195,size=9).encode(alt.X('start:T',scale=alt.Scale(domain=brush)),text='evento',color=alt.value('#000000'))

pts2=alt.selection(type="single",encodings=['x'])
bar2 = alt.Chart('data/topicos.json').mark_bar().encode(alt.X('Partido:N'),\
                                                        alt.Y('median(Word_Count):Q',stack='center',axis=None),color=alt.condition(pts2, alt.ColorValue("red"), alt.ColorValue("grey"))).\
properties(width=500,height=50).transform_filter(pts).add_selection(pts2)

pts3=alt.selection(type="single",encodings=['x'])
bar3 = alt.Chart('data/topicos.json').mark_bar().encode(alt.X('Partido:N'),\
                                                        alt.Y('median(Word_Count):Q',stack='center',axis=None),color=alt.condition(pts3, alt.ColorValue("red"), alt.ColorValue("grey"))).\
properties(width=500,height=50).transform_filter(pts).add_selection(pts3)

pts4=alt.selection(type="single",encodings=['y'])

bar4 = alt.Chart('data/topicos.json').mark_rect().encode(alt.X('Dominant_Topic:N',scale=alt.Scale(domain=pts)),\
                                                        alt.Y('Nome:N',axis=alt.Axis(orient='right')),color=alt.Color('Word_Count:Q',scale=alt.Scale(scheme='plasma'),legend=alt.Legend(orient='bottom')),tooltip=[alt.Tooltip('Nome:N',title='Nome'),\
                                                                                                                                                  alt.Tooltip('Partido:N',title='Partido')]).\
properties().transform_filter(pts2).add_selection(pts4).transform_filter(pts)#.properties(width=500,height=5000)

upper2= base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush)),alt.Color('Dominant_Topic:N',legend=None,scale=alt.Scale(scheme='dark2')),tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]).transform_filter(pts2).transform_filter(pts).properties(width=500,height=200)#.transform_filter(pts4)


pts5=alt.selection(type="single",encodings=['y'])
bar5 = alt.Chart('data/topicos.json').mark_rect().encode(alt.X('Dominant_Topic:N'),\
                                                        alt.Y('Nome:N'),color=alt.Color('Word_Count:Q',scale=alt.Scale(scheme='plasma'),legend=alt.Legend(orient='bottom')),tooltip=[alt.Tooltip('Nome:N',title='Nome'),\
                                                                                                                                                  alt.Tooltip('Partido:N',title='Partido')]).\
properties().transform_filter(pts3).add_selection(pts5).transform_filter(pts)#.properties(width=500,height=5000)


upper3= base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush)),alt.Y('sum(Word_Count):Q',stack='center',axis=None),alt.Color('Dominant_Topic:N',legend=None,scale=alt.Scale(scheme='dark2')),tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]).transform_filter(pts3).transform_filter(pts).properties(width=500,height=200)#.transform_filter(pts5)#.configure_legend(orient='bottom')



text_display=alt.Chart('data/topicos.json').mark_text(align='left',baseline='top',dy=-195,size=9).encode(alt.X('Data:T'),alt.Y('Nome:N'),text='Text:N',color=alt.value('#000000'))


total_chart = bar & (rule+text+upper) & (rule+lower) & (bar2|bar3) & alt.hconcat(upper2 , upper3).resolve_scale(y='shared')# & alt.hconcat(bar4, bar5,spacing=200)#.transform_sample(20) #& text_display.transform_sample(20)

In [25]:
total_chart.save('teste.html')

In [58]:
total_chart

alt.VConcatChart(...)